In [3]:
import zipfile
import os

# # Create the destination folder
os.makedirs("dataset/real", exist_ok=True)

# # Unzip the CelebA zip file
with zipfile.ZipFile("img_align_celebA.zip", 'r') as zip_ref:
     zip_ref.extractall("dataset/real")

print("CelebA images extracted to dataset/real")


CelebA images extracted to dataset/real


In [1]:
import zipfile
import os

# # Create the destination folder for fake images
os.makedirs("dataset/fake", exist_ok=True)

# # Unzip the SFHQ zip file
with zipfile.ZipFile("images.zip", 'r') as zip_ref:
     zip_ref.extractall("dataset/fake")

print("SFHQ images extracted to dataset/fake")

SFHQ images extracted to dataset/fake


In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())


1.7.1+cu101
True


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image
import numpy as np

# Check device whether it is working in gpu environment or not
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Dataset path
dataset_path = './dataset'

# Resizing and Normalization
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10)
])
# Data Transformations

train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])



Using device: cuda


In [3]:
# Step 1: Gather image paths and labels manually
class_names = sorted(os.listdir(dataset_path))  # Example: ['real', 'fake']
image_paths = []
labels = []

for idx, class_name in enumerate(class_names):
    class_dir = os.path.join(dataset_path, class_name)
    for img_name in os.listdir(class_dir):
        image_paths.append(os.path.join(class_dir, img_name))
        labels.append(idx) #ASSIGN LABELS (Fake : 0 and Real:1)

image_paths = np.array(image_paths)
labels = np.array(labels)
print(image_paths)
print(labels)

['./dataset\\fake\\SFHQ_pt3_00000001.jpg'
 './dataset\\fake\\SFHQ_pt3_00000002.jpg'
 './dataset\\fake\\SFHQ_pt3_00000003.jpg' ...
 './dataset\\real_and_fake\\SFHQ_pt3_00070540.jpg'
 './dataset\\real_and_fake\\SFHQ_pt3_00070541.jpg'
 './dataset\\real_and_fake\\SFHQ_pt3_00070542.jpg']
[0 0 0 ... 2 2 2]


In [4]:
# Step 2: Select only 15000 real and 15000 fake images
real_indices = np.where(labels == 0)[0][:15000]
fake_indices = np.where(labels == 1)[0][:15000]

selected_indices = np.concatenate((real_indices, fake_indices))
np.random.shuffle(selected_indices)

selected_image_paths = image_paths[selected_indices]
selected_labels = labels[selected_indices]
print(selected_image_paths)
print(selected_labels)

['./dataset\\real\\002897.jpg' './dataset\\fake\\SFHQ_pt3_00004966.jpg'
 './dataset\\fake\\SFHQ_pt3_00009424.jpg' ...
 './dataset\\fake\\SFHQ_pt3_00004543.jpg' './dataset\\real\\008976.jpg'
 './dataset\\real\\007205.jpg']
[1 0 0 ... 0 1 1]


In [5]:
# Step 3: Split into training and validation sets (80-20 split)
split = int(0.8 * len(selected_indices))
train_image_paths = selected_image_paths[:split]
train_labels = selected_labels[:split]
val_image_paths = selected_image_paths[split:]
val_labels = selected_labels[split:]

In [6]:
# Step 4: Custom Dataset to resize only selected images
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

In [7]:
train_dataset = CustomImageDataset(train_image_paths, train_labels, transform=train_transform)
val_dataset = CustomImageDataset(val_image_paths, val_labels, transform=val_transform)


In [8]:
# Step 6: DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=0, pin_memory=True)

In [9]:
# Model Setup
# model = models.squeezenet1_1(pretrained=True)
# model.classifier[1] = nn.Conv2d(512, 2, kernel_size=(1, 1), stride=(1, 1))
# model.num_classes = 2
# model = model.to(device)


In [10]:
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, 2)
model.num_classes = 2
model = model.to(device)

In [11]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.log_softmax = nn.LogSoftmax(dim=-1)

    def forward(self, x, target):
        pred = self.log_softmax(x)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=-1))


In [12]:
# Loss and Optimizer
criterion = LabelSmoothingLoss(classes=2, smoothing=0.1)
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [13]:
# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device, non_blocking=True), labels.long().to(device, non_blocking=True)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct.double() / len(train_loader.dataset)

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')
        validate_model(model, val_loader)


In [14]:
#Validation function
def validate_model(model, val_loader):
    model.eval()
    correct = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device, non_blocking=True), labels.long().to(device, non_blocking=True)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)

    val_acc = correct.double() / len(val_loader.dataset)
    print(f'Validation Accuracy: {val_acc:.4f}')

In [15]:
# Start Training
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=15)

Epoch 1/15, Loss: 0.3527, Accuracy: 0.9886
Validation Accuracy: 1.0000
Epoch 2/15, Loss: 0.3334, Accuracy: 0.9992
Validation Accuracy: 1.0000
Epoch 3/15, Loss: 0.3294, Accuracy: 0.9998
Validation Accuracy: 1.0000
Epoch 4/15, Loss: 0.3279, Accuracy: 0.9999
Validation Accuracy: 1.0000
Epoch 5/15, Loss: 0.3270, Accuracy: 1.0000
Validation Accuracy: 1.0000
Epoch 6/15, Loss: 0.3268, Accuracy: 0.9997
Validation Accuracy: 1.0000
Epoch 7/15, Loss: 0.3264, Accuracy: 0.9999
Validation Accuracy: 1.0000
Epoch 8/15, Loss: 0.3262, Accuracy: 0.9998
Validation Accuracy: 1.0000
Epoch 9/15, Loss: 0.3263, Accuracy: 0.9996
Validation Accuracy: 1.0000
Epoch 10/15, Loss: 0.3258, Accuracy: 0.9999
Validation Accuracy: 1.0000
Epoch 11/15, Loss: 0.3259, Accuracy: 0.9998
Validation Accuracy: 1.0000
Epoch 12/15, Loss: 0.3255, Accuracy: 1.0000
Validation Accuracy: 1.0000
Epoch 13/15, Loss: 0.3255, Accuracy: 1.0000
Validation Accuracy: 1.0000
Epoch 14/15, Loss: 0.3255, Accuracy: 1.0000
Validation Accuracy: 1.0000
E

In [16]:
#save model parameters, model states
torch.save(model.state_dict(), 'model_updated_3.pth')

In [17]:
# New Cell to Export Model to ONNX

import torch
import torch.nn as nn
from torchvision import models

# Ensure the model architecture matches your trained model
# This is crucial for successful ONNX export
model = models.mobilenet_v2(pretrained=False) # No need for pretrained weights during export
model.classifier[1] = nn.Linear(model.last_channel, 2) # Adjust final layer for 2 classes
model.num_classes = 2 # Set num_classes for consistency

# Load the trained weights
model.load_state_dict(torch.load('model_updated_3.pth'))
model.eval() # Set the model to evaluation mode for inference

# Define a dummy input tensor with a FIXED batch size of 1
# This is the key change for the ONNX export
# Shape: [batch_size, channels, height, width]
dummy_input = torch.randn(1, 3, 128, 128) # 1 image, 3 color channels, 128x128 pixels

# Define the ONNX export path
onnx_model_path = "model_updated_3.onnx"

# Export the model to ONNX
torch.onnx.export(
    model,
    dummy_input,
    onnx_model_path,
    export_params=True,        # Export trained parameter weights inside the model file
    opset_version=11,          # The ONNX opset version to use (11 is common and widely supported)
    do_constant_folding=True,  # Whether to execute constant folding for optimization
    input_names=['input'],     # The name to assign to the input node in the ONNX graph
    output_names=['output'],   # The name to assign to the output node in the ONNX graph
    # NO dynamic_axes for the batch dimension (index 0)
    # If you had dynamic_axes={'input': {0: 'batch_size'}}, remove it for fixed batch size.
)

print(f"Model successfully exported to {onnx_model_path} with fixed batch size 1.")

Model successfully exported to model_updated_3.onnx with fixed batch size 1.
